In [25]:
using DataFrames, XLSX, DataStructures, Random, LinearAlgebra

In [ ]:
xf = XLSX.readxlsx("velib_data_simulation.xlsx")
data = xf["Feuille1"]

parser = x -> if x isa String parse(Float64, x) elseif x isa Number Float64(x) else NaN end

# NOTE that the stations will be shifted : stations are 3:7 in the sheet, but 1:5 in the code
# station 1 in the code correspond to station 3 in the sheets, ...
# General data
routing = parser.(data["B4:F8"])

# EVERYTHING IS IN MINUTES : average number of departures per minute  
departures = vec(parser.(data["B13:F13"])) ./ 60

M = length(departures)

traversal_times = convert.(Float64, parser.(data["B18:F22"]))

# Initial condiditons
docks_per_station = vec(data["B26:F26"]) # Should not be used

initial_bikes = convert.(Int, vec(data["B30:F30"]))
sum(initial_bikes) # Souldn't it be 100 ? Oh well anyways the initial state should not matter anyways


initial_in_transit = parser.(data["B34:F38"])
replace!(x -> isnan(x) ? 0 : x, initial_in_transit)
initial_in_transit = convert.(Int, initial_in_transit)

routing

5×5 Matrix{Float64}:
 0.0   0.2   0.3   0.2   0.3
 0.2   0.0   0.3   0.2   0.3
 0.2   0.25  0.0   0.25  0.3
 0.15  0.2   0.3   0.0   0.35
 0.2   0.25  0.35  0.2   0.0

## Analyse théorique


In [27]:
matrix = zeros(M, M)

for i ∈ 1:M
    matrix[i, i] = departures[i]
    for j ∈ 1:M
        if j == i
            continue
        end
        matrix[i, j] = - departures[j] * routing[j, i]
    end
end

kernel = nullspace(matrix)

alpha_stations_non_normalised = kernel[:, 1]

alphas_non_normalised = zeros(M, M)

for i ∈ 1:M
    alphas_non_normalised[i, i] = alpha_stations_non_normalised[i]
    for j ∈ 1:M
        if j == i 
            continue
        end
        alphas_non_normalised[i, j] = alpha_stations_non_normalised[i] * departures[i] * routing[i, j] * traversal_times[i, j]
    end
end

alphas_normalised = alphas_non_normalised ./ sum(alphas_non_normalised)

probas_stations_are_empty = [1 - alphas_normalised[i, i] for i ∈ 1:M]

5-element Vector{Float64}:
 0.8245327221069128
 0.8444956475598524
 0.8654753784142575
 0.8443318448057511
 0.8407257694851767

## Simulateur  

In [28]:
# Sampling sanity check 
SAMPLES = 1000000


# Test using StatsBase
using StatsBase
counts_1 = zeros(Int, 5)
@time begin
    for _ ∈ 1:SAMPLES
        res = sample(1:5, Weights(@view routing[1, :]))
        counts_1[res] += 1
    end
end

# Using a custom sampler
"""
Given a sorted vector of thresholds that represent [p1, p1+p2, ..., p1 + ... + pn-1, p1 + ... + pn = 1],
LAST ELEMENT HAS TO BE 1
Return a random int in 1:n with probabilities of p1 for 1, ...
"""
function custom_sample(thresholds :: Vector{Float64})
    u = rand()
    res = findfirst(x -> (x > u), thresholds)
    return res
end

counts_2 = zeros(Int, 5)
thresholds = [sum(routing[1, 1:i]) for i ∈ 1:5]

@time begin
    for _ ∈ 1:SAMPLES
        res = custom_sample(thresholds)
        counts_2[res] += 1
    end
end

# Sanity check
println("Draw frequencies 1 : $(counts_1 ./ SAMPLES)")
println("Draw frequencies 2 : $(counts_2 ./ SAMPLES)")

# Fastest is custom sampler with precomputed thresholds !

  0.918668 seconds (7.99 M allocations: 213.545 MiB, 3.05% gc time)
  0.186308 seconds (4.99 M allocations: 91.475 MiB, 10.11% gc time)
Draw frequencies 1 : [0.0, 0.199923, 0.299857, 0.200278, 0.299942]
Draw frequencies 2 : [0.0, 0.199947, 0.300222, 0.200103, 0.299728]


In [29]:
# Exponential draw sanity check
SAMPLES = 1000000

# Test using Distributions.jl
using Distributions
distrib = Exponential(1 / departures[1])
res_1 = []
@time begin
    for _ ∈ 1:SAMPLES
        res = rand(distrib)
        push!(res_1, res)
    end
end

# Test using a custom function
function draw_exponential(lambda :: Float64)
    u = rand()
    return (-1 / lambda) * log(u)
end

res_2 = []
@time begin
    for _ ∈ 1:SAMPLES
        res = draw_exponential(departures[1])
        push!(res_2, res)
    end
end

# Sanity check
println("Average for first distrib : $(sum(res_1) / SAMPLES), for second : $(sum(res_2) / SAMPLES) ")

# We see that the custom hand made one is a little bit faster, or near enough. Will will thus only use custom function !

  0.395872 seconds (4.00 M allocations: 104.578 MiB, 13.48% gc time)
  0.353464 seconds (5.00 M allocations: 119.839 MiB, 20.43% gc time)
Average for first distrib : 21.39746902046965, for second : 21.44397723611261 


In [30]:
# Next departure in at t, with time = t + X where X ~ Exp(lambda)
struct Departure
    time :: Float64
    origin :: Int
end

# At a departure time, draw the destination using the routing matrix, and set the arrival to t + travel_time[origin, dest]
# Or should travel time also be drawn with an exponential ? Might be ?
struct Arrival
    time :: Float64
    origin :: Int
    destination :: Int
end

Event = Union{Departure, Arrival};

In [31]:
function simulator(
    departures :: Vector{Float64},
    routing_probas :: Matrix{Float64},
    traversal_times :: Matrix{Float64},
    initial_states :: Vector{Int},
    initial_in_transit :: Matrix{Int},
    Tmax :: Float64,
    verbose :: Bool = false,
    seed :: Int = 42,
)
    Random.seed!(seed)
    t = 0
    N = sum(initial_states) + sum(initial_in_transit)
    M = length(initial_states)

    event_queue = PriorityQueue{Event, Float64}(Base.Order.Forward)

    # Initial departures
    for i ∈ 1:M
        # Draws a random time with the given distribution
        t_dep = draw_exponential(departures[i])
        event_queue[Departure(t_dep, i)] = t_dep
    end

    # Initial arrivals for the bikes initially in transit
    for i ∈ 1:M
        for j ∈ 1:M
            n_in_transit = initial_in_transit[i, j]
            for k ∈ 1:n_in_transit
                t_arr = draw_exponential(1 / traversal_times[i, j])
                event_queue[Arrival(t_arr, i, j)] = t_arr
            end
        end 
    end

    # Pre-build the threshold vectors used for sampling destinations
    routing_thresholds = [[sum(routing[i, 1:j]) for j ∈ 1:M] for i ∈ 1:M]
    # Correct possible floating point errors where probas might not exactly add to 1.0
    for i ∈ 1:M
        routing_thresholds[i][end] = 1.0
    end

    state = copy(initial_states)
    journeys_in_progress = copy(initial_in_transit)

    iter = 0

    # Statistics 
    
    # Counting with Poisson arrivals
    empty_when_we_try_departure = zeros(Int, M) # Count the number of times a station is empty before we try to depart from it
    # Counting with average
    last_departures = zeros(Float64, M) # Keep track of the last time there was a departure from each station.
    time_spent_at_zero = zeros(Float64, M) # When a bike arrives at an empty station, we thus know how long it spent empty
    departures_per_station = zeros(Int, M) # Count the number of departures we try at each station
    

    while t < Tmax
        # Events are dequeued in increasing order of event.time
        event = dequeue!(event_queue)
    
        if event isa Departure
            t = event.time
            origin = event.origin
            departures_per_station[origin] += 1
            if state[origin] > 0
                # Pick the destination and create the arrival event
                dest = custom_sample(routing_thresholds[origin])
                t_arrival = t + draw_exponential(1 / traversal_times[origin, dest])
                # Add the Arrival event to the queue
                event_queue[Arrival(t_arrival, origin, dest)] = t_arrival
                journeys_in_progress[origin, dest] += 1
                state[origin] -= 1
                last_departures[origin] = t
            else
                # Journey is lost, do something ?
                empty_when_we_try_departure[origin] += 1
            end

            # In any case, draw the next departure event
            t_next = t + draw_exponential(departures[origin])
            # Add the next Departure from this station to the queue
            event_queue[Departure(t_next, origin)] = t_next

        elseif event isa Arrival
            t = event.time
            ori, dest = event.origin, event.destination
            # If we arrive at empty station, the station spent (t - last_departures[dest]) minutes empty !
            if state[dest] == 0
                time_spent_at_zero[dest] += (t - last_departures[dest])
            end
            # Simply update the states (We do not consider station capacity !)
            journeys_in_progress[ori, dest] -= 1
            state[dest] += 1
        end

        iter += 1
    end

    lost_prop = empty_when_we_try_departure ./ departures_per_station
    average_at_zero = time_spent_at_zero ./ t

    if verbose
        println("Reached Tmax = $Tmax, $iter events treated")
        println("Sanity check : number of bikes is $(sum(state) + sum(journeys_in_progress))")
        println("----------------------------")
        println("Proportion of departures lost due to empty station : $lost_prop")
        println("Proportion of the time spent empty : $average_at_zero")
        println("Gap : $(abs.(lost_prop - average_at_zero))")
    end
    
    return state, lost_prop, average_at_zero
end


simulator (generic function with 3 methods)

## Simulation avec 1 vélo


In [ ]:
# It take around 2min to run a single simulation on my machine with Tmax_Single = 5e8
Tmax_Single = 5e8

# Define initial state for a single bike
single_bike_stations = zeros(Int, M)
single_bike_stations[1] = 1
single_bike_initial_transit = zeros(Int, M, M)


simulator(
    departures,
    routing,
    traversal_times,
    single_bike_stations,
    single_bike_initial_transit,
    Tmax_Single,
    true
);

Reached Tmax = 5.0e8, 193212586 events treated
Sanity check : number of bikes is 1
----------------------------
Proportion of departures lost due to empty station : [0.824575916113731, 0.8445105704484117, 0.8655066807608992, 0.8444128606782488, 0.8406977907853803]
Proportion of the time spent empty : [0.8244401274947394, 0.8444989182376303, 0.8655170027632068, 0.8443853190563705, 0.8406601930954571]
Gap : [0.00013578861899166128, 1.1652210781365469e-5, 1.0322002307550093e-5, 2.7541621878346412e-5, 3.759768992317447e-5]


In [ ]:
# It takes around 1min30 to run 1000 parallel simulations with Tmax_Multiple = 1e6 on my machine
Tmax_Multiple = 1e6
N_simus = 1000

simus_ending_at_zero_single_bike = zeros(Int, M)

Threads.@threads for i ∈ 1:N_simus
    state, _, __ = simulator(   
        departures,
        routing,
        traversal_times,
        single_bike_stations,
        single_bike_initial_transit,
        Tmax_Multiple,
        false,
        i
    )
    for (i, s) in enumerate(state)
        if s == 0
            simus_ending_at_zero_single_bike[i] += 1
        end
    end
end

# We compute the proportion of simulation where each station ends with zero bikes 

println("Proportion of simulation where each station ends with zero bikes : $(simus_ending_at_zero_single_bike ./ N_simus)")

Proportion of simulation where each station ends with zero bikes : [0.823, 0.841, 0.88, 0.866, 0.867]


We want to give an estimate and its confidence interval of the average time each station spends with 0 bikes stored

### Confidence interval

The confidence interval of level $1 - \alpha$ is computed using the formula :
$$ I_{1-\alpha} = \left[ \bar{X} - t_{\alpha/2}^{r-1} \frac{s}{\sqrt{r}} ; \bar{X} + t_{\alpha/2}^{r-1} \frac{s}{\sqrt{r}} \right] $$

Where : $t_{\alpha/2}^{r-1}$ is the quantile of the t-distribution with r-1 degrees of freedom at $\alpha/2$, $\bar{X}$ is the sample mean, $s$ is the sample standard deviation and $r$ is the number of simulation runs.

In [34]:
using Distributions
student_t = TDist(N_simus - 1)
alpha = 0.05 # 95% confidence intervalls

function analyse_multiple_simulations(number_ending_at_zeros :: Vector{Int})
    for (i, tot) in enumerate(number_ending_at_zeros)
        # We are sampling 1_{X_i^T == 0}, which always either one or zero.
        # The variance is thus 1 / N * (tot * (1 - avg)² + (N - tot) * avg²)
        avg = tot / N_simus
        dev = sqrt(
            (1 / N_simus) * (tot * (1 - avg)^2 + (N_simus - tot) * (avg)^2 )
        )
        quant = quantile(student_t, alpha / 2)
        width = quant * (dev / sqrt(N_simus))

        # println("Tot is $tot, Avg is $avg, std dev is $dev")
        print("Asymptotic proportion of time station $i spends empty is : ")
        println("$avg ± $width")
    end
end

analyse_multiple_simulations(simus_ending_at_zero_single_bike)

Asymptotic proportion of time station 1 spends empty is : 0.823 ± -0.023684331477431604
Asymptotic proportion of time station 2 spends empty is : 0.841 ± -0.022691914574234164
Asymptotic proportion of time station 3 spends empty is : 0.88 ± -0.02016538597361345
Asymptotic proportion of time station 4 spends empty is : 0.866 ± -0.021139072346394887
Asymptotic proportion of time station 5 spends empty is : 0.867 ± -0.021072203372849402


## Simulation à 91 vélos (selon l'état inital dans le fichier)

In [35]:
println("Number of bikes initially in system according to file : $(sum(initial_bikes) + sum(initial_in_transit))")

Number of bikes initially in system according to file : 91


In [ ]:
# It take around 3min to run a single simulation on my machine with Tmax_Single = 5e8
Tmax_Single = 5e8

simulator(
    departures,
    routing,
    traversal_times,
    initial_bikes,
    initial_in_transit,
    Tmax_Single,
    true
);

Reached Tmax = 5.0e8, 313909115 events treated
Sanity check : number of bikes is 91
----------------------------
Proportion of departures lost due to empty station : [0.0003363739556132945, 0.11388665947270413, 0.23369310773719676, 0.11290634699142454, 0.09282817498937598]
Proportion of the time spent empty : [0.00033395676926431447, 0.11382854396792241, 0.23373442468779967, 0.1129794241126312, 0.09278563198085228]
Gap : [2.41718634898003e-6, 5.81155047817139e-5, 4.13169506029154e-5, 7.307712120666054e-5, 4.254300852370163e-5]


In [ ]:
# It takes around 2min30 to run 1000 parallel simulations with Tmax_Multiple = 1e6 on my machine
Tmax_Multiple = 1e6
N_simus = 1000

simus_ending_at_zero = zeros(Int, M)

Threads.@threads for i ∈ 1:N_simus
    state, _, __ = simulator(   
        departures,
        routing,
        traversal_times,
        initial_bikes,
        initial_in_transit,
        Tmax_Multiple,
        false,
        i
    )
    for (i, s) in enumerate(state)
        if s == 0
            simus_ending_at_zero[i] += 1
        end
    end
end

# We compute the proportion of simulation where each station ends with zero bikes 

println("Proportion of simulation where each station ends with zero bikes : $(simus_ending_at_zero ./ N_simus)")

Proportion of simulation where each station ends with zero bikes : [0.0, 0.103, 0.246, 0.122, 0.089]
